In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
pd.options.mode.chained_assignment = None

/Users/alirezafaghaninia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
test = pd.read_csv('../data/test.csv', usecols=['item_id'])

In [3]:
periods = pd.read_csv('periods.csv')

In [4]:
activation_date = pd.read_csv('activation_date.csv')

In [5]:
tabular = pd.read_csv('tabular.csv')

In [6]:
nlp = pd.read_csv('nlp.csv')

In [7]:
test['periods_weight'] = 0.0
test['tabular_weight'] = 5.0
test['nlp_weight'] = 0.0
test['activation_date_weight'] = 0.0

In [8]:
len(test)

508438

In [9]:
test.isna().sum() / len(test)

item_id                   0.0
periods_weight            0.0
tabular_weight            0.0
nlp_weight                0.0
activation_date_weight    0.0
dtype: float64

In [10]:
test = test.merge(periods, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'periods'})
test['periods_weight'][null_idx] = 0.0
test['periods'][null_idx] = 0.0

In [11]:
test = test.merge(tabular, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'tabular'})
test['tabular_weight'][null_idx] = 0.0
test['tabular'][null_idx] = 0.0

In [12]:
test = test.merge(nlp, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'nlp'})
test['nlp_weight'][null_idx] = 0.0
test['nlp'][null_idx] = 0.0

In [13]:
test = test.merge(activation_date, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'activation_date'})
test['activation_date_weight'][null_idx] = 0.0
test['activation_date'][null_idx] = 0.0

In [14]:
test['deal_probability'] = test['periods']*test['periods_weight'] + \
                            test['tabular']*test['tabular_weight'] + \
                            test['nlp']*test['nlp_weight'] + \
                            test['activation_date']*test['activation_date_weight'] 
test['deal_probability'] /= ( test['periods_weight'] + test['tabular_weight'] + test['nlp_weight'] + test['activation_date_weight'])

In [15]:
submission = test[['item_id', 'deal_probability']].set_index('item_id')

In [16]:
### methods for transformation to [0, 1] range:
### 1) just cut out the negative part:
submission[submission < 0] = 0.0

### 2) MinMaxScaler to [0,1] range
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# submission['deal_probability'] = scaler.fit_transform(submission)

submission.min()

deal_probability    0.001718
dtype: float64

In [17]:
submission.to_csv('submission.csv')

In [18]:
len(submission)

508438

In [19]:
submission.head()

,deal_probability
item_id,
6544e41a8817,0.053102
65b9484d670f,0.360651
8bab230b2ecd,0.188644
8e348601fefc,0.362879
8bd2fe400b89,0.218020
